In [26]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/aremath/.local/lib/python3.10/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['Infinity']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [27]:
# Item Rando:
# 26 'randomizeable' items
# 100 'randomizeable' drop locations
# idea 1: location : item_id
# ~ 500 bits (locations * log(items))
# idea 2: item : location_id
# ~ 700 bits (locations * log(locations))
# idea 3: location : common_item_id
# major_item : location_id
# 300 bits + 126 bits

In [28]:
# Door Rando:
# ~550 doors
# idea 1: door: door_id
# 550 * log(550) = 5500 bits
# idea 2: left_door : right_door_id
# top_door : bottom_door_id
# 225 * log(225) = 2040
# idea 3:
# X, Y
# overworld is ~ 128 x 128
# 14 bits per room * 255 rooms = 3570

In [29]:
import sys
sys.path.append("../..")

In [30]:
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms

In [31]:
from functools import cache
from itertools import combinations

In [32]:
rooms = parse_rooms("../../encoding/dsl/rooms_for_alloy.txt")
exits = parse_exits("../../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(rooms, exits)

In [33]:
from rom_tools.rom_manager import RomManager
from abstraction_validation.sm_paths import *

In [34]:
from bdds.bdd_core import *
from bdds.node_bdds import *

In [35]:
from functools import reduce

In [36]:
all_nodes = []
for r, room in rooms.items():
    for node in room.graph.name_node.keys():
        all_nodes.append(node)
node_ids = {n:i for i,n in enumerate(all_nodes)}

In [37]:
# Issue with design translation via chopping off the last _
node_ids["Spore_Spawn_Spawn"] = node_ids["Spore_Spawn_Spore_Spawn"]
node_ids["Golden_Torizo_Torizo"] = node_ids["Golden_Torizo_Golden_Torizo"]
node_ids["Mother_Brain_Brain"] = node_ids["Mother_Brain_Mother_Brain"]

In [38]:
major_items = ["MB", "B", "SPB", "G", "SA", "V", "GS", "SB", "HJ", "CB", "WB", "PLB", "Spazer", "XR", "IB", "SJ"]
minor_items = ["M", "S", "PB", "E", "RT"]
rando_items = ["B", "MB", "PB", "SPB", "S", "M", "G", "SA", "V", "GS", "SB", "HJ", "CB", "WB", "E", "PLB", "Spazer", "RT", "XR", "IB", "SJ"]
assert set(rando_items) == set(major_items) | set(minor_items)

In [39]:
len(major_items)

16

In [40]:
len(rando_items)

21

In [41]:
context_id = mk_context_id(node_ids)

In [42]:
def loc_id(room_name, node_name, when="prev"):
    node_id = node_ids[f"{room_name}_{node_name}"]
    return f"node_id_{when} = {node_id}"

#TODO: make this more efficient
@cache
def item_transitions(item_gained=None):
    if item_gained is None:
        return "(items_unchanged)"
    clauses = []
    for i in design["Items"] | design["Bosses"]:
        if i == item_gained:
            clause = f"{i}_prev < {i}_next"
        else:
            clause = f"{i}_prev = {i}_next"
        clauses.append(clause)
    return " & ".join(clauses)

def itemset_to_str(itemset):
    if len(itemset) == 0:
        return "TRUE"
    else:
        return " & ".join([f"{item}_prev = 1" for item in itemset])

def required_itemsets(itemsets):
    return "(" + " | ".join([itemset_to_str(itemset) for itemset in itemsets]) + ")"

def rando_transitions(room_name, node_name, family, possible_items):
    t = context_id.false
    node_name = room_name + "_" + node_name
    drop_name = f"{family}_drop_{node_name}"
    for i,item in enumerate(possible_items):
        t |= context_id.add_expr(f"{drop_name} = {i}") & context_id.add_expr(item_transitions(item))
        #t |= context.add_expr(f"{drop_name} = {i} &" + item_transitions(item))
    return t

In [43]:
major_nodes = []
minor_nodes = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        if d in major_items:
            major_nodes.append(room_name + "_" + node_name)
        if d in minor_items:
            minor_nodes.append(room_name + "_" + node_name)

In [44]:
assert len(major_nodes) == len(major_items)
assert len(minor_nodes) == 100 - len(major_nodes)

In [45]:
drop_nodes = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        if d in rando_items:
            drop_nodes.append(room_name + "_" + node_name)

In [46]:
assert len(drop_nodes) == 100

In [47]:
major_item_nodes = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        if d in rando_items:
            drop_nodes.append(room_name + "_" + node_name)

In [48]:
#limit design freedom by setting a limited number of locations
#drop_nodes = ["Bomb_Torizo_Bombs", "Big_Pink_M2"]
n_major_rando = 16
n_minor_rando = 0
major_drop_nodes = major_nodes[:n_major_rando]
minor_drop_nodes = minor_nodes[:n_minor_rando]
#drop_nodes = []

In [49]:
design_vars = {
    **{ f"major_drop_{node}": (0,len(major_items)-1) for node in major_drop_nodes },
    **{ f"minor_drop_{node}": (0,len(minor_items)-1) for node in minor_drop_nodes }
}

In [50]:
if len(design_vars) > 0:
    context_id.declare(**design_vars)

In [51]:
# Ensure uniqueness and validity
validity = context_id.true
for node in major_drop_nodes:
    validity &= context_id.add_expr(f"major_drop_{node} < {len(major_items)}")
for node in minor_drop_nodes:
    validity &= context_id.add_expr(f"minor_drop_{node} < {len(major_items)}")

# Only drop one thing
#uniqueness = context_id.true
#for node, node2 in combinations(major_drop_nodes, 2):
#    uniqueness &= context_id.add_expr(f"major_drop_{node} != major_drop_{node2}")
# Sum is faster to compute than O(n^2) !=s
uniqueness = context_id.true
for i,item in enumerate(major_items):
    uniqueness &= context_id.add_expr(" + ".join([f"ite(major_drop_{node} = {i},1,0)" for node in major_drop_nodes] + ["0"]) + " <= 1")

valid_design = validity & uniqueness

In [52]:
original_design = context_id.true
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        name = room_name + "_" + node_name
        if d in major_items:
            if name in major_drop_nodes:
                original_design &= context_id.add_expr(f"major_drop_{name} = {major_items.index(d)}")
        if d in minor_items:
            if name in minor_drop_nodes:
                original_design &= context_id.add_expr(f"minor_drop_{name} = {minor_items.index(d)}")

In [53]:
edits = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        name = room_name + "_" + node_name
        if d in major_items:
            if name in major_drop_nodes:
                edits.append(f"ite(major_drop_{name} = {major_items.index(d)},0,1)")
        if d in minor_items:
            if name in minor_drop_nodes:
                edits.append(f"ite(minor_drop_{name} = {minor_items.index(d)},0,1)")
all_edits = " + ".join(edits)
max_distance = 4
edit_distance = context_id.add_expr(all_edits + f" <= {max_distance}")
edit_distance_equal = context_id.add_expr(all_edits + f" = {max_distance}")

In [54]:
ed = edit_distance & valid_design

In [55]:
ed.count()

17621.0

In [56]:
import math

In [57]:
ed_test = ed & context_id.add_expr("major_drop_Bomb_Torizo_Bombs = 0")

In [58]:
#TODO: minor items
def show_design(pick):
    for v,id in pick.items():
        print(f"{v}: {major_items[id]}")

In [59]:
show_design(context_id.pick(ed & edit_distance_equal))

major_drop_Bomb_Torizo_Bombs: MB
major_drop_Big_Pink_CB: HJ
major_drop_Morph_Ball_Room_MB: B
major_drop_XRay_XR: XR
major_drop_Spazer_Spazer: Spazer
major_drop_Varia_Suit_V: V
major_drop_Ice_Beam_IB: IB
major_drop_Hi_Jump_HJ: CB
major_drop_Grapple_G: G
major_drop_Speed_Booster_SB: SB
major_drop_Wave_Beam_WB: WB
major_drop_Screw_Attack_SA: SA
major_drop_Gravity_GS: GS
major_drop_Plasma_PLB: PLB
major_drop_Spring_Ball_SPB: SPB
major_drop_Space_Jump_SJ: SJ


In [60]:
# 1, 121, 1241, 17621

In [61]:
# 1 , 2 , 9, 44 - Rencontres number
# |edit_distance_equal_n| = (16 choose n) * r_n, where r_n is the nth Rencontres number
# |edit_distance_n| = sum(|edit_distance_equal_k|) from k=0 to k=n

In [62]:
def combine_bdd_group(bdds):
    return reduce(lambda x,y: x|y, bdds, context_id.false)

In [63]:
# Build individual BDDs
room_bdds = []
door_bdds = []
for room_name, room in tqdm(design['Rooms'].items()):
    links = []
    for node_name in room['Nodes']:
        s = loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions()
        links.append(context_id.add_expr(s, with_ops=True))
        if node_name in room['Drops']:
            if room_name + "_" + node_name in major_drop_nodes:
                s = context_id.add_expr(loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next")) & rando_transitions(room_name, node_name, "major", major_items)
            elif room_name + "_" + node_name in minor_drop_nodes:
                s = context_id.add_expr(loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next")) & rando_transitions(room_name, node_name, "minor", minor_items)
            else:
                s = context_id.add_expr(loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions(room['Drops'][node_name]), with_ops=True)
            links.append(s)
    for node_name, door in room['Doors'].items():
        d = loc_id(room_name, node_name) + " & " + loc_id(door['Room'], door['Node'], when="next") + " & " + item_transitions()
        door_bdds.append(context_id.add_expr(d, with_ops=True))
    for node_name, edges in room['Edges'].items():
        for edge in edges:
            other_node_name = edge['Terminal']
            s = loc_id(room_name, node_name) + " & " + loc_id(room_name, other_node_name, when="next") + " & " + required_itemsets(edge['Requirements']) + " & " + item_transitions()
            links.append(context_id.add_expr(s, with_ops=True))
    room_bdd = combine_bdd_group(links)
    #print(room_bdd.count())
    room_bdds.append(room_bdd)

100%|█████████████████████████████████████████████████████████████████████████████████| 255/255 [00:14<00:00, 18.14it/s]


In [64]:
doors_bdd = combine_bdd_group(door_bdds)
rooms_bdd = combine_bdd_group(room_bdds)
all_bdd = doors_bdd | rooms_bdd

In [65]:
all_bdd.dag_size

28896

In [66]:
all_bdd.count()
context_id.exist(prevs, all_bdd).count()

349.0

In [67]:
len(all_bdd.support)

156

In [68]:
items_unchanged = context_id.add_expr("items_unchanged", with_ops=True)

In [69]:
item_nexts = [k for k in item_vars if k.endswith("_next")]

In [70]:
def get_reachable_nav_steps(trans, context, start_bdd, reorder=True):
    n = 0
    #trans_local = context_id.exist(item_nexts + ["Ceres_Ridley_next"], all_bdd & ~context_id.add_expr("node_id_prev = node_id_next"))
    trans_local = context.exist(item_nexts, trans & items_unchanged)
    #trans_local = trans & items_unchanged
    covered = start_bdd
    covered_last = context.false
    while covered != covered_last:
        if n < 3 and reorder:
            context.bdd.reorder()
        covered_last = covered
        # Inner while loop for no-pickup navigation
        covered_local = covered
        covered_local_last = context.false
        while covered_local != covered_local_last:
            covered_local_last = covered_local
            fringe_local = context.exist(["node_id_prev"], covered_local & trans_local)
            covered_local |= context.let({"node_id_next": "node_id_prev"}, fringe_local)
            #print(context.support(covered_local))
            print("\t", covered_local.dag_size)
        covered |= covered_local
        # Unrestricted step
        fringe = context.exist(prevs, covered & trans)
        covered |= context.let(next_to_prev, fringe)
        print(n, covered.dag_size)
        n+=1
    return covered

In [71]:
def get_reachable_id(trans_norule, context, start_bdd):
    n = 0
    covered_p = start_bdd
    covered_last_p = context.false
    while covered_p != covered_last_p:
        covered_last_p = covered_p
        fringe_n = context.exist(prevs, covered_p & trans_norule)
        covered_p |= context.let(next_to_prev, fringe_n)
        print(n, covered_p.dag_size)
        n+=1
    return covered_p

In [72]:
def mk_closure(trans, context, max_steps=float("inf")):
    n = 0
    closure = trans
    closure_last = context.false
    while closure != closure_last and n < max_steps:
        closure_last = closure
        closure_prev_temp = context.let(next_to_temp, closure_last)
        closure_temp_next = context.let(prev_to_temp, closure_last)
        closure |= context.exist(temps, closure_prev_temp & closure_temp_next)
        print(n, closure.dag_size)
        n+=1
    closure_square = closure
    return closure_square

In [73]:
def get_reachable_nav_closure(trans, context, start_bdd):
    n = 0
    #trans_local = context_id.exist(item_nexts + ["Ceres_Ridley_next"], all_bdd & ~context_id.add_expr("node_id_prev = node_id_next"))
    trans_local = trans & items_unchanged
    trans_local = mk_closure(trans_local, context)
    trans = trans_local | trans
    #trans_local = trans & items_unchanged
    covered = start_bdd
    covered_last = context.false
    while covered != covered_last:
        covered_last = covered
        fringe = context.exist(prevs, covered & trans)
        covered |= context.let(next_to_prev, fringe)
        print(n, covered.dag_size)
        n+=1
    return covered

In [74]:
from data_types import item_set

In [75]:
def mk_itemset_expr(itemset, when="prev"):
    clauses = []
    for i in item_mapping:
        if i in itemset:
            clause = f"{i}_{when} = 1"
        else:
            clause = f"{i}_{when} = 0"
        clauses.append(clause)
    return " & ".join(clauses)

In [76]:
start_bdd = context_id.add_expr("node_id_prev = 0 & " + mk_itemset_expr(item_set.ItemSet()))

In [77]:
all_items_bad = context_id.true
for drop in design_vars.keys():
    all_items_bad &= context_id.add_expr(f"{drop} < 4")

In [78]:
context_id.bdd.configure()

{'reordering': False,
 'garbage_collection': True,
 'max_memory': 18446744073709551615,
 'loose_up_to': 6710886,
 'max_cache_soft': 11497472,
 'max_cache_hard': 4294967295,
 'min_hit': 30,
 'max_growth': 1.2,
 'max_swaps': 2000000,
 'max_vars': 1000}

In [79]:
#%%time
#reachable2 = get_reachable_nav_steps(all_bdd, context_id, start_bdd & valid_design & original_design)

In [80]:
#context_id.exist(nexts, reachable2).count()

In [81]:
%%time
all_bdd_fast = mk_closure(all_bdd, context_id, 3)

0 51391
1 123339
2 448511
CPU times: user 2.84 s, sys: 18.3 ms, total: 2.86 s
Wall time: 2.85 s


In [82]:
%%time
reachable = get_reachable_nav_steps(all_bdd, context_id, start_bdd & valid_design & edit_distance, reorder=False)
#reachable = get_reachable_nav_steps(all_bdd_fast, context_id, start_bdd & valid_design & edit_distance, reorder=False)
#reachable = get_reachable_nav_steps(all_bdd_fast, context_id, start_bdd)
#reachable = get_reachable_nav_closure(all_bdd, context_id, start_bdd)

	 38093
	 38095
	 38096
	 38104
	 38106
	 38105
	 38105
	 38105
	 38106
	 38111
	 38113
	 38113
	 38113
0 41700
	 41700
	 41701
	 41719
	 41724
	 41749
	 41748
	 41749
	 41750
	 41752
	 41760
	 41725
	 41727
	 41729
	 41729
	 41727
	 41729
	 41728
	 41728
	 41728
1 41766
	 41794
	 41796
	 41852
	 41869
	 41910
	 41880
	 41844
	 41851
	 41853
	 41858
	 41859
	 41846
	 41844
	 41821
	 41796
	 41801
	 41803
	 41801
	 41805
	 41804
	 41805
	 41807
	 41807
2 42134
	 42148
	 42149
	 42179
	 42183
	 42198
	 42189
	 42190
	 42167
	 42170
	 42170
	 42172
	 42171
	 42167
	 42150
	 42151
	 42139
	 42145
	 42142
	 42144
	 42147
	 42146
	 42144
	 42144
3 42487
	 42489
	 42491
	 42530
	 42537
	 42546
	 42546
	 42554
	 42550
	 42552
	 42553
	 42553
	 42544
	 42539
	 42536
	 42544
	 42538
	 42565
	 42566
	 42562
	 42565
	 42564
	 42566
	 42565
	 42549
	 42549
	 42549
	 42553
	 42553
4 42669
	 42718
	 42740
	 42794
	 42773
	 42858
	 42851
	 42852
	 42815
	 42799
	 42800
	 42768
	 42756
	 42771
	 42772


In [83]:
#%%time
#reachable3 = get_reachable_id(all_bdd, context_id, start_bdd & valid_design & edit_distance)

In [84]:
#context_id.bdd.dump("../output/item_designs_ed6.dddmp", [reachable])

In [85]:
#%%time
#context_id.bdd.reorder()

CPU times: user 1min 20s, sys: 263 ms, total: 1min 20s
Wall time: 1min 19s


In [86]:
len(context_id.bdd.vars)

248

In [95]:
reachable.count()

29792165364.0

In [96]:
reachable.dag_size

381685

In [88]:
designs = context_id.exist(nexts + prevs, reachable)

In [89]:
designs.count()

17621.0

In [91]:
end_reachable = context_id.add_expr("node_id_prev = 1")

In [98]:
(reachable & end_reachable).dag_size

42512

In [92]:
good_designs = context_id.exist(nexts + prevs, reachable & end_reachable)

In [93]:
# ~3.3% of designs are good at ED6
good_designs.count() / designs.count()

0.11384143919187334

In [94]:
good_designs.count()

2006.0

In [ ]:
good_designs.dag_size

In [ ]:
major_drop_nodes

In [ ]:
#place = "Varia_Suit_V"
place = "Big_Pink_CB"
d = {}
for i,item in enumerate(major_items):
    ismorph = context_id.add_expr(f"major_drop_{place} = {i}")
    count = (ismorph & good_designs).count()
    if count > 0:
        print(item, count)
        d[item] = count

In [ ]:
# If you select a design uniformly at random there are 76325 possibilities
# If you select an item for Varia_Suit_V UAR, what is the effective number of possibilities?

In [ ]:
len(d)

In [ ]:
n = 0
for i in d.values():
    n += i
print(n)

In [ ]:
n_designs = good_designs.count()

In [ ]:
# View X (the design) as a discrete RV.
# Entropy(X) = - sum(log p(x))
# -log_2 (1/76325) = log_2(76325) = 16.22 -> uniform distribution is ~16 bits of info

In [ ]:
q = 0
for i in d.values():
    q -= (i / n_designs) * math.log(i / n_designs, 2)
print(q)

In [ ]:
q2 = 0
for i in d.values():
    print(math.log(i, 2))
    q2 += (i / n_designs) * math.log(i, 2)
print(q2)

In [ ]:
q + q2

In [ ]:
# VERSUS

In [ ]:
q = math.log(len(d), 2)

In [ ]:
q2 = 0
for i in d.values():
    q2 += (1 / len(d)) * math.log(i, 2)
print(q2)

In [ ]:
q + q2

In [ ]:
math.log(n_designs, 2)

In [ ]:
math.log(n_designs, 2) - (q + q2)

In [ ]:
n_designs

In [ ]:
#context_id.bdd.dump("../../output/item_designs_ed6.dddmp", [good_designs])

In [ ]:
# add to trans 1.844674407372149e+19
# add to start 11939936.0
# 11939936.0

In [ ]:
#%%time
#reachable = get_reachable_id(all_bdd_fast, context_id, start_bdd)

In [ ]:
#TODO: Iterative Squaring

In [ ]:
g_wo_v = context_id.add_expr("GS_prev = 1 & V_prev = 0")
v_wo_g = context_id.add_expr("GS_prev = 0 & V_prev = 1")

In [ ]:
(end_reachable & reachable).count()

In [ ]:
#TODO: ensure no invalid node ids

In [ ]:
get_g_wo_v = context_id.exist(prevs, reachable & g_wo_v)
no_get_v_wo_g = ~context_id.exist(prevs, reachable & v_wo_g)
get_end = context_id.exist(prevs, reachable & end_reachable)

In [ ]:
(get_g_wo_v & no_get_v_wo_g & get_end).dag_size

In [ ]:
(get_g_wo_s & get_end).count()

In [ ]:
v_at_sj = context_id.add_expr(f"major_drop_Gravity_GS = {major_items.index('V')}")

In [ ]:
(get_g_wo_v & no_get_v_wo_g & v_at_sj & get_end).count()

In [ ]:
for sol, _ in zip(context_id.pick_iter(get_g_wo_v & no_get_v_wo_g & v_at_sj & get_end), range(10)):
    td = {}
    for k,v in sol.items():
        if v < len(major_items):
            td[k] = major_items[v]
        else:
            td[k] = v
    print(td)

In [ ]:
# Gaussian distribution over shortest path times
# Action n-grams
# Major/Minor rando
# nx.draw layout

In [ ]:
#TODO: Region-based or item-collection based cuts
# Saturate the space ahead-of-time